## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-26-23-41-40 +0000,startribune,Naked man found in Senate chamber at State Cap...,https://www.startribune.com/man-found-in-state...
1,2025-07-26-23-40-48 +0000,nyt,11 People Injured in Stabbing at a Michigan Wa...,https://www.nytimes.com/2025/07/26/us/stabbing...
2,2025-07-26-23-23-14 +0000,nypost,NY AG Letitia James condemns Gaza ‘humanitaria...,https://nypost.com/2025/07/26/us-news/letitia-...
3,2025-07-26-23-15-15 +0000,nypost,Doctor’s note won’t save sick kids from court ...,https://nypost.com/2025/07/26/us-news/doctors-...
4,2025-07-26-23-12-33 +0000,bbc,Nigeria seal 10th Wafcon as comeback stuns Mor...,https://www.bbc.com/sport/football/articles/c1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
112,trump,29
33,new,14
17,gaza,9
116,out,9
135,aid,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
76,2025-07-26-16-09-07 +0000,nypost,"Trump mediates Cambodia-Thailand conflict, dem...",https://nypost.com/2025/07/26/us-news/trump-me...,65
28,2025-07-26-21-07-33 +0000,nyt,Trump Says He Expects Quick Cease-Fire Between...,https://www.nytimes.com/2025/07/26/world/asia/...,58
146,2025-07-26-02-25-06 +0000,nyt,New Reports on Russian Interference Show Trump...,https://www.nytimes.com/2025/07/25/us/trump-ob...,57
97,2025-07-26-12-48-43 +0000,cbc,Fighting on Thai-Cambodia border continues for...,https://www.cbc.ca/news/world/thailand-cambodi...,56
123,2025-07-26-09-30-00 +0000,wsj,Now that President Trump’s One Big Beautiful B...,https://www.wsj.com/economy/trump-tax-cuts-win...,56


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
76,65,2025-07-26-16-09-07 +0000,nypost,"Trump mediates Cambodia-Thailand conflict, dem...",https://nypost.com/2025/07/26/us-news/trump-me...
157,45,2025-07-26-01-00-00 +0000,wsj,The Democratic Party’s image has eroded to its...,https://www.wsj.com/politics/elections/democra...
58,33,2025-07-26-18-05-36 +0000,bbc,Gaza aid site offered a 'women only' day. It d...,https://www.bbc.com/news/articles/c74z4gy5g31o
69,29,2025-07-26-16-54-10 +0000,nypost,Ukraine fends off major Russian advance on fro...,https://nypost.com/2025/07/26/world-news/ten-c...
123,27,2025-07-26-09-30-00 +0000,wsj,Now that President Trump’s One Big Beautiful B...,https://www.wsj.com/economy/trump-tax-cuts-win...
11,26,2025-07-26-22-52-47 +0000,latimes,City by the Bay? More like City by the Brrr! S...,https://www.latimes.com/california/story/2025-...
148,26,2025-07-26-02-23-22 +0000,nypost,Democratic Texas congressional candidate dragg...,https://nypost.com/2025/07/25/us-news/democrat...
113,26,2025-07-26-11-00-00 +0000,wsj,Faced with a rising number of threats and shak...,https://www.wsj.com/politics/policy/congress-s...
21,22,2025-07-26-21-55-57 +0000,nypost,Iranian Supreme Leader sleeps and gets ‘high o...,https://nypost.com/2025/07/26/world-news/irani...
2,21,2025-07-26-23-23-14 +0000,nypost,NY AG Letitia James condemns Gaza ‘humanitaria...,https://nypost.com/2025/07/26/us-news/letitia-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
